In [45]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from api_keys import g_key


In [46]:
#Import weather data

weather_df = pd.read_csv("..\\WeatherPy\\weather_data.csv")
weather_df.head()

,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind_Speed,Hemisphere
0,Busselton,-33.65,115.33,52.79,52,0,12.15,Southern
1,Cidreira,-30.18,-50.21,66.56,86,100,19.62,Southern
2,Yellowknife,62.46,-114.35,23.00,85,90,9.17,Northern
3,Ushuaia,-54.80,-68.30,48.20,61,75,21.92,Southern
4,Okakarara,-20.58,17.43,94.57,10,45,11.63,Southern


In [47]:
#Store variables to be used in map

locations = weather_df[["Latitude", "Longitude"]].astype(float)
humidity = weather_df["Humidity"].astype(float)

In [67]:
#Create map
gmaps.configure(api_key=g_key)

figure = gmaps.figure()
humidity_heatmap = gmaps.heatmap_layer(locations, weights=humidity, 
                                       dissipating=False, max_intensity=100, 
                                       point_radius = 2.5)

figure.add_layer(humidity_heatmap)

figure

Figure(layout=FigureLayout(height='420px'))

In [49]:
humidity

0       52.0
1       86.0
2       85.0
3       61.0
4       10.0
       ...  
500     64.0
501     78.0
502     22.0
503     74.0
504    100.0
Name: Humidity, Length: 505, dtype: float64

In [50]:
#Narrow down dataframe

vacation_df = weather_df.loc[(weather_df["Temperature"] < 85) & (weather_df["Temperature"] > 70) & (weather_df["Wind_Speed"] < 10) & (weather_df["Cloudiness"] < 10)]
vacation_df = vacation_df.dropna()
vacation_df.count()

City           30
Latitude       30
Longitude      30
Temperature    30
Humidity       30
Cloudiness     30
Wind_Speed     30
Hemisphere     30
dtype: int64

In [61]:
coords_list = []
for index, row in vacation_df.iterrows():
    coords = str(row['Latitude']) + "," + str(row['Longitude'])
    coords_list.append(coords)


vacation_df["Locations"] = coords_list
coords_list

['-34.42,19.23',
 '-6.88,112.21',
 '27.53,68.76',
 '-6.84,-79.93',
 '25.12,62.33',
 '17.49,44.13',
 '36.59,-79.39',
 '14.9,-24.5',
 '11.28,-15.83',
 '22.83,78.82',
 '-23.35,47.6',
 '-24.79,-50.01',
 '-20.32,118.57',
 '24.69,46.72',
 '37.06,9.24',
 '35.47,44.39',
 '21.39,-158.15',
 '29.45,74.65',
 '6.8,81.48',
 '30.92,74.6',
 '-19.98,57.61',
 '36.98,38.42',
 '29.16,16.14',
 '30.3,76.46',
 '21.35,-158.09',
 '-33.73,-70.75',
 '31.45,12.98',
 '14.91,105.08',
 '-0.87,134.08',
 '-20.01,57.66']

In [60]:
#Work with Google APIs to get hotels

hotels_list = []
new_coords = []
for index, row in hotel_df.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params = {
        "location": row["Locations"],
        "types": "lodging",
        "radius": 5000,
        "key": g_key
    }
    response = requests.get(base_url, params)
    hotels = response.json()["results"]
    if hotels:
        hotels_list.append(hotels[0]["name"])
        new_coords.append(row["Locations"])

hotels

[{'business_status': 'OPERATIONAL',
  'geometry': {'location': {'lat': -20.0065691, 'lng': 57.66807650000001},
   'viewport': {'northeast': {'lat': -20.0053813197085,
     'lng': 57.66935138029151},
    'southwest': {'lat': -20.0080792802915, 'lng': 57.6666534197085}}},
  'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
  'name': 'Veranda Paul et Virginie Hotel & Spa',
  'opening_hours': {'open_now': True},
  'photos': [{'height': 3024,
    'html_attributions': ['<a href="https://maps.google.com/maps/contrib/107085669206390457321">Tamara Oberholster</a>'],
    'photo_reference': 'CmRaAAAAqVEeW_Xv5qvpPTaEjI_1vWAWsi9y-UsdA20kBcwcSiHTniPn1IoP6IZqHWlQmGJtOps73JrU5B4ElAsIquwi9t1YfR9AguTNoOwKqmB2pvN99nAN-GoydT0e1eUVMSr4EhCVtYUTMXFI7hgqKAUZHArtGhRvs-KrvmXHuhbodiCf-gPjtolWog',
    'width': 4032}],
  'place_id': 'ChIJ983FouoAfSEREtMMToedgo4',
  'plus_code': {'compound_code': 'XMV9+96 Grand Gaube, Mauritius',
   'global_code': '5HFVXMV9+96'},
  'rating': 4.5,

In [68]:
hotel_df = pd.DataFrame({"Hotel Name": hotels_list, "Locations": new_coords})
hotel_df = pd.merge(hotel_df, vacation_df, on='Locations', how='left')
hotel_df

,Hotel Name,Locations,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind_Speed,Hemisphere
0,Misty Waves Boutique Hotel,"-34.42,19.23",Hermanus,-34.42,19.23,71.01,69,0,5.99,Southern
1,bador sepret,"-6.88,112.21",Labuhan,-6.88,112.21,81.63,76,7,1.07,Southern
2,Khaskheli house Wafa Fida,"27.53,68.76",Khairpur,27.53,68.76,75.20,69,0,8.43,Northern
3,Casona Del Muelle,"-6.84,-79.93",Pimentel,-6.84,-79.93,73.40,60,0,4.70,Southern
4,Sadaf Resort,"25.12,62.33",Gwadar,25.12,62.33,80.17,42,0,7.94,Northern
5,محمد هادي ال فطيح,"17.49,44.13",Najrān,17.49,44.13,80.60,11,0,9.17,Northern
6,Holiday Inn Express Danville,"36.59,-79.39",Danville,36.59,-79.39,71.60,78,1,1.57,Northern
7,Tortuga B&B,"14.9,-24.5",São Filipe,14.90,-24.50,82.47,69,5,9.51,Northern
8,"Casa Dora, Bubaque","11.28,-15.83",Bubaque,11.28,-15.83,83.14,78,7,8.90,Northern
9,जयश टेंट हाउस चीचली,"22.83,78.82",Chichli,22.83,78.82,73.24,48,4,4.52,Northern


In [78]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

info_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info
)

figure.add_layer(info_layer)

In [79]:
figure

Figure(layout=FigureLayout(height='420px'))